In [1]:
from string import digits 
import re
import pandas as pd
import pyodbc

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")

In [2]:
cnxn = pyodbc.connect('DRIVER={Oracle in OraClient11g_home1};')

In [3]:
def stemText(m1text):
    text = list(filter(None, re.split('\s|,|:|;|-', m1text)))
    nonDots = map(lambda each:each.replace(".", '').replace("(", '').replace(")", '').replace("№", ''), text)
    nonNums = map(lambda c: re.sub(r'\d+', '', c), nonDots) 
    
    l=[stemmer.stem(word) for word in nonNums]
    finalWords = list(filter(lambda c: len(c) > 2, l))
    return list(set(finalWords))

In [ ]:
def runModel(m1data, m3):
    if len(m1data.index) != 0:
        m1data.dropna(subset=['LONGTEXT'], inplace=True)
        m1data["stems"] = m1data.apply(lambda x: stemText(x[1]), axis = 1)

        noMsausCount = m1data[m1data["MSAUS"] == 0].count()
        MsausCount = m1data[m1data["MSAUS"] == 1].count()
        
        if (noMsausCount.QMNUMQ < 4 or MsausCount.QMNUMQ < 4):
            cursor = cnxn.cursor()
            cursor.execute("insert into sm_mess_analysis_res (qmnumq, decision, m1s, score) values (?, ?, ?, ?);", [qm, -1.0, -1.0, 0.0])
            cnxn.commit()
        else:
            noMsausData = m1data[m1data["MSAUS"] == 0].head(min(noMsausCount.QMNUMQ, MsausCount.QMNUMQ))
            MsausData = m1data[m1data["MSAUS"] == 1].head(min(noMsausCount.QMNUMQ, MsausCount.QMNUMQ))

            m1data = pd.concat([noMsausData, MsausData])

            m1sentences = m1data["stems"].apply(' '.join)

            vectorizer = CountVectorizer(min_df=0.1, max_df=0.85, lowercase=False)
            vectorizer.fit(m1sentences)
            vectorizer.transform(m1sentences)

            y = m1data['MSAUS'].values

            sentences_train, sentences_test, y_train, y_test = train_test_split(m1sentences, y, test_size=0.2, random_state=1000)
            
            vectorizer.fit(sentences_train)

            X_train = vectorizer.transform(sentences_train)
            X_test  = vectorizer.transform(sentences_test)

            classifier = LogisticRegression()

            model = classifier.fit(X_train, y_train)
            score = round(model.score(X_test, y_test), 4)

            result = model.predict(vectorizer.transform([longtext]))

            cursor = cnxn.cursor()
            cursor.execute("insert into sm_mess_analysis_res (qmnumq, decision, m1s, score) values (?, ?, ?, ?);", [m3, result[0], float(y.size), score])
            cnxn.commit()
    else:
        cursor = cnxn.cursor()
        cursor.execute("insert into sm_mess_analysis_res (qmnumq, decision, m1s, score) values (?, ?, ?, ?);", [m3, -1.0, 0.0, 0.0])
        cnxn.commit()

In [ ]:
m3s = pd.read_sql_query("""select d.qmnumq from d_m_in d
                           inner join vso_iflo vi on vi.tplnrq = d.tplnrq
                           where d.qmart = 'M3'
                            and d.ausvn > sysdate - 2
                            and d.longtext is not null
                            and length(d.longtext) > 15
                            and d.qmnumq not in (select qmnumq from sm_mess_analysis_res)
                            and d.ausbs is null
                            and vi.rktgtplnrq is not null
                           """, cnxn)

for index, row in m3s.iterrows():
    m3data = pd.read_sql_query("""select v.qmnumq, v.headktxt, d.longtext, vipar.tplmaq as tplnrq 
                              from vso_viqmel v
                              inner join d_m_in d on d.qmnumq = v.qmnumq 
                              inner join vso_iflo vi on vi.tplnrq = v.tplnrq
                              inner join vso_iflo vipar on vipar.tplnrq = vi.tplmaq
                              where v.qmnumq = ?
                           """, cnxn, params=[row['QMNUMQ']])

    tp = m3data.iloc[0]['TPLNRQ']
    qm = m3data.iloc[0]['QMNUMQ']
    longtext = ' '.join(stemText(m3data.iloc[0]['LONGTEXT']))

    sql = """SELECT d.qmnumq, d.longtext, case when substr(v.qmgrp, -1) = '6' 
                                      then 1 else 0 end as msaus
                  FROM VSO_VIQMEL v
                      INNER JOIN D_M_IN d ON v.qmnumq = d.qmnumq
                  WHERE v.qmart = 'M1' 
                  AND length(d.longtext) > 15
                  AND lower(d.longtext) NOT LIKE '%разбор%в%сообщении%'
                  AND lower(d.longtext) NOT LIKE '%разбор%произведен%'
                  AND v.tplnrq IN (
                      SELECT tplnrq FROM VSO_IFLO
                      START with tplnrq = ? 
                      CONNECT BY PRIOR tplnrq = tplmaq
                  )
       """

    m1s = pd.read_sql(sql, cnxn, params=[tp])
    runModel(m1s, qm)

C:\Users\ma.mishutushkin\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
